## Special Token Operations
#### 04.4 Winter School on Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2021-11-28


## Setup
Starting with this chapter 4.1, the lines below will always automatically load ...
* The functions in `algo_util.py`
* The accounts MyAlgo, Alice and Bob
* The Purestake credentials

In [ ]:
# Loading shared code and credentials
import sys, os
codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the 5 main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [ ]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.future.transaction import PaymentTxn
from algosdk.future.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
import algosdk.error
import json

In [ ]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])

## Modifications of an ASA
A few properties of an ASA can never be modified
* Name and unit name
* Decimals
* Total supply

Using the special roles, it is however possible to ...
* Freeze/unfreeze ASA assets of a specific address
* Claim ASA back from a specific address
* Destroy an ASA
* Change the special roles


### First cretate Tempcoin
To study the whole life of an ASA, we create a new one, called `Tempcoin`, with the symbol `TEMP` (for temporary), with the following roles:

* **Manager** = Alice
* **Reserve** = Bob
* **Freeze** = Charlie
* **Clawback** = Dina (this will change later to Alice)

In [ ]:
# Step 1: Prepare
sp = algod_client.suggested_params()
token_supply = 100                               # Token supply
token_decimals =  1                              # How many digits after the comma?
token_total = token_supply * 10**token_decimals  # Specify SMALLER unit ("cents")

token_name  = "Peters Tempcoin"                  # <----- YOUR NAME HERE
token_url   = "en.wikipedia.org/wiki/Temporary"  # <----- CHANGE if you want to
token_unitname = "TEMP"

# Step 2: Asset creation tansaction
txn = AssetConfigTxn(
    sender=MyAlgo['public'],                   # Creator of the ASA
    sp=sp,                                     # Network parameters
    total=token_total,                         # Token supply in SMALL unit
    decimals=token_decimals,
    default_frozen=False,                      
    unit_name=token_unitname,                       
    asset_name=token_name,
    url=token_url,
    manager=Alice['public'],                   # Special roles
    reserve=Bob['public'],
    freeze=Charlie['public'],
    clawback=Dina['public'], 
)

# Step 3: Sign and send
stxn = txn.sign(MyAlgo['private'])             # Sign
txid = algod_client.send_transaction(stxn)     # Send
print(txid)

# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client,txid)

In [ ]:
# Step 5: asset_id and information
TEMP_id = txinfo['asset-index']
print(TEMP_id)
print('https://testnet.algoexplorer.io/asset/{}'.format(TEMP_id))

In [ ]:
asset_holdings_df(algod_client,MyAlgo['public'])

## Use the Reserve address
Transfer 50% to the reserve address, so that they are shown as "not yet minted"

#### Opt-in transaction of Bob

In [ ]:
# Even though Bob is the reserve, he has to opt in

# Step 1: prepare and create TX
sp = algod_client.suggested_params()
amt = int(0)                              # <------ opt-in = 0 transaction

txn = AssetTransferTxn(
    sender=Bob['public'],                 # <------- From Bob ...
    sp=sp,
    receiver=Bob['public'],               # <------- ... to Bob
    amt=amt,
    index=TEMP_id)                        # <----- Correct index TEMP_id

# Step 2: sign and send
stxn = txn.sign(Bob['private'])           # <----- Signed by Bob
txid = algod_client.send_transaction(stxn)
print(txid)

# Step 3: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

#### Transfer to the reserve (= Bob)

In [ ]:
# Step 1: prepare and create TX
sp = algod_client.suggested_params()
amt = int(token_total / 2)                # <------- half of the holdings
txn = AssetTransferTxn(
    sender=MyAlgo['public'],
    sp=sp,
    receiver=Bob['public'],               # <------- Bob is the reserve
    amt=amt,
    index=TEMP_id)                        # <------- Correct index TEMP_id

# Step 2: sign and send
stxn = txn.sign(MyAlgo['private'])         # <----- Signed by the creator of the asset
txid = algod_client.send_transaction(stxn)
print(txid)

# Step 3: wait for confirmation
wait_for_confirmation(algod_client, txid)

In [ ]:
asset_holdings_df(algod_client,Bob['public'])

In [ ]:
# Check "Circulating Supply" in Algoexplorer
print('https://testnet.algoexplorer.io/asset/{}'.format(TEMP_id))

## Clawback a transaction
* Normally, only Bob could authorize a transfer of his tokens ... except for the clawback address `Dina`.
* She claws back 25 tokens from Bob
* No opt-in necessary for clawback, because she does not receive any coins

In [ ]:
sp = algod_client.suggested_params()

# Must be signed by the account that is the Asset's clawback address
txn = AssetTransferTxn(
    sender=Dina['public'],                  # <---- Clawback = Dina is sender of transaction
    sp=sp,
    receiver=MyAlgo["public"],              # <---- Money goes back to MyAlgo
    amt=25*10,                              # <---- Amount in SMALL units
    index=TEMP_id,
    revocation_target=Bob['public']         # <---- Take the money out of Bob's account
    )
stxn = txn.sign(Dina['private'])            # <---- Signed by Clawback = Dina
txid = algod_client.send_transaction(stxn)
print(txid)

# Wait for the transaction to be confirmed
txinfo = wait_for_confirmation(algod_client, txid)

In [ ]:
# Check Bob's asset holdings
asset_holdings_df(algod_client,Bob['public'])

## Reconfigure the asset
* A reconfiguration is changing any of the four special roles
* The **Manager** (Alice) can reconfigure the asset. 
* In a reconfiguration, all four roles must again be specified
* No opt-in is necessary for reconfigure, because no coins are receved

**Example**
* Alice removes the clawback role from Dina and assigns it to herself
* Same `AssetConfigTxn` as when creating an asset

In [ ]:
# Step 1: Prepare
sp = algod_client.suggested_params()

# Step 2: Create Asset reconfiguration tansaction
txn = AssetConfigTxn(
    sender=Alice['public'],                    # Manager
    sp=sp,                                     # Network parameters
    index=TEMP_id,                             # <------------ MUST specify the asset ID
    manager=Alice['public'],                   # Old role, repeated
    reserve=Bob['public'],                     # Old role, repeated
    freeze=Charlie['public'],                  # Old role, repeated
    clawback=Alice['public'],                  # <-------- New responsibiliy
)
#print(txn)

# Step 3: Sign and send
stxn = txn.sign(Alice['private'])              # <---- Signed by the manager!
txid = algod_client.send_transaction(stxn)     # Send
print(txid)

# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client,txid)

In [ ]:
# Open in Algoexplorer: manager and clawback are now the same
print('https://testnet.algoexplorer.io/asset/{}'.format(TEMP_id))

**TRY THIS** If Dina tries to clawback the other half of Bob's holdings, it will not work

## Freeze assets

#### Charlie freezes Bob's assets
* The freeze role (= Charlie) freezes the asset holdings of Bob.
* New transaction type `AssetFreezeTxn`
* No opt-in necessary for freezing

In [ ]:
# Step 1: Prepare
sp = algod_client.suggested_params()

# Step 2: Asset freeze tansaction
txn = AssetFreezeTxn(
    sender=Charlie['public'],                # <---- Must be initiated by freeze address
    sp=sp,
    index=TEMP_id,
    target=Bob["public"],                    # <---- This is the address that we want to freeze  
    new_freeze_state=True   
    )

# Step 3: Sign and send
stxn = txn.sign(Charlie['private'])          # <---- Must be signed by the freeze role(=Charlie)
txid = algod_client.send_transaction(stxn)
print(txid)

# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client,txid)

#### Bob wants to spend some TEMP token
* Bob tries to send some TEMP token to Alice, but he cannot do it
* Only his TEMP token are frozen

In [ ]:
# Step 1: prepare and create TX
sp = algod_client.suggested_params()
amt = int(5 * 10)    

txn = AssetTransferTxn(
    sender=Bob['public'],                 # <------- Bob wants to send ...
    sp=sp,
    receiver=Alice['public'],             # <------- ... to Alice  
    amt=amt,
    index=TEMP_id)                        # <----- asset_id for TEMP token

# Step 2+3: sign and send
stxn = txn.sign(Bob['private'])           # <----- Signed by Bob

try:
    txid = algod_client.send_transaction(stxn)
except algosdk.error.AlgodHTTPError as err:
    # print entire error message
    print(err)
    if ("frozen" in str(err)):                # check for specific type of error
        print("Asset is frozen")         
    txid = None
    
# Step 4: Wait for confirmation
# There is no step 4 here, because we already obtain an error on step 3

#### Exercise: unfreezeing Bobs tokens
* How can we unfreeze Bobs tokens? 
* The operation is similar to freezing, except for ...

In [ ]:
# Your Python code goes here ...




## Destroy asset
An asset can only be destroyed, if all coins are back in the creator's accounn (MyAlgo). Then the manager (Alice) can destroy the asset.

### Return all TEMP coins to MyAlgo and opt out
* Use `AssetTransferTxn` to send back coins
* Addtionally we have to opt out, using `close_assets_to`
    * *Closing* means sending the remaining balance to a specific address

In [ ]:
# Step 1: prepare and create TX
sp = algod_client.suggested_params()
amt = int(25 * 10)    

txn = AssetTransferTxn(
    sender=Bob['public'],                 
    sp=sp,
    receiver=MyAlgo['public'],            
    amt=amt,
    index=TEMP_id,                        
    close_assets_to=MyAlgo['public']      # <------- Opt out of the asset 
    )                       

# Step 2+3: sign and send
stxn = txn.sign(Bob['private'])           # <----- Signed by Bob
txid = algod_client.send_transaction(stxn)
    
# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client,txid)

In [ ]:
# Open in Algoexplorer to check holders
print('https://testnet.algoexplorer.io/asset/{}'.format(TEMP_id))

### Destroying the asset
* Use a special `AssetConfigTxn` transaction

In [ ]:
# Step 1: prepare
sp = algod_client.suggested_params()

# Step 2: Create special AssetConfigTxn to destroy
txn = AssetConfigTxn(
    sender=Alice['public'],              # Manager must be sender
    sp=sp,
    index=TEMP_id,
    total = None,
    strict_empty_address_check=False     # include this option in destroy ops
    )

#Step 2: Send 
stxn = txn.sign(Alice['private'])

# Step 3: Send the transaction and check for errors
try:
    txid = algod_client.send_transaction(stxn)
except algosdk.error.AlgodHTTPError as err:
    print(err)                                              # print entire error message
    if ("cannot destroy asset" in str(err)):                # check for specific type of error
        print("Cannot destroy asset {}, not holding all tokens.".format(TEMP_id))  
    if ("does not exist or has been deleted" in str(err)): 
        print("Cannot destroy asset. It does not exist (any more).")    

In [ ]:
# Step 4: Wait for the transaction to be confirmed
wait_for_confirmation(algod_client, txid)

In [ ]:
# Now check the "assets created" for that account.
account_info = algod_client.account_info(MyAlgo['public'])
json_str = json.dumps(account_info['created-assets'])
json.loads(json_str)

**EXERCISE** Check the holdings of MyAlgo, Alice and Bob. Check the asset-id on Algoexplorer.

## Things that don't work

* Holding an ASA if you do not hold sufficient ALGOs
    * Minimum 0.1 ALGO per account plus 0.1 ALGO per ASA
* Creating more that 1000 ASA per address
* Destroying an ASA if the creator does not hold all coins